# Electricity Price Data Preprocessing for Clustering and ML

**Input**: Raw hourly price data from 5 largest European zones

**Output**: Clean 24-hour daily profiles ready for clustering and XGBoost modeling

## Scientific Methodology

Following validated preprocessing methods from peer-reviewed literature:

1. **Outlier Detection**: Z-score (threshold=3) - Chikodili et al. (2021)
2. **Missing Data**: Cubic spline interpolation - Moritz & Bartz-Beielstein (2017)
3. **Normalization**: Min-max [0,1] and z-score - Faiq et al. (2024)
4. **Framework**: Systematic preprocessing - Wang et al. (2023)

## Zones Processed

- **ES**: Spain
- **PL**: Poland
- **NO2**: Norway (Southwest, largest)
- **SE3**: Sweden (Central, largest)
- **DK1**: Denmark (West, largest)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
import warnings
from scipy.stats import zscore
warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10
sns.set_style('whitegrid')

print("Libraries loaded successfully")

In [ ]:
# Directory structure
DATA_DIR = Path('data')
RAW_DIR = DATA_DIR / 'raw'
CLEAN_DIR = DATA_DIR / 'clean'
PROFILES_DIR = DATA_DIR / 'daily_profiles'
FIGURES_DIR = DATA_DIR / 'figures'

CLEAN_DIR.mkdir(exist_ok=True, parents=True)
PROFILES_DIR.mkdir(exist_ok=True, parents=True)
FIGURES_DIR.mkdir(exist_ok=True, parents=True)

# Zone configuration (largest per country)
ZONES = ['ES', 'PL', 'NO2', 'SE3', 'DK1']

COUNTRY_MAP = {
    'ES': 'Spain',
    'PL': 'Poland',
    'NO2': 'Norway',
    # 'SE2': 'Sweden',  # Backup zone
    'DK1': 'Denmark'
}

print(f"Configuration: {len(ZONES)} zones")
print(f"Countries: {', '.join(COUNTRY_MAP.values())}")

## Data Loading

In [ ]:
def load_zone_data(zone_name):
    """
    Load raw price data from collection step
    """
    filepath = RAW_DIR / f"{zone_name}_raw.csv"
    
    if not filepath.exists():
        print(f"Warning: {filepath} not found")
        return None
    
    df = pd.read_csv(filepath)
    df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)
    
    return df

# Load all zones
print("Loading raw data...\n")
all_data = {}

for zone in ZONES:
    df = load_zone_data(zone)
    if df is not None:
        all_data[zone] = df
        print(f"{zone}: {len(df):,} hourly records")

print(f"\nLoaded: {len(all_data)}/{len(ZONES)} zones")

## Preprocessing Pipeline

### Step 1: Outlier Detection (Z-Score Method)

**Reference**: Chikodili et al. (2021) - Outlier Detection in Multivariate Time Series Data

**Method**: 
- Calculate z-score for each price observation
- Flag outliers: |z-score| > 3.0
- Replace outliers with NaN for interpolation

**Rationale**: 
- F-measure: 0.9978 (superior to IQR method's 0.8571)
- Standard approach in electricity price analysis
- Probability < 0.003 for points beyond ±3 standard deviations

### Step 2: Missing Data Imputation (Cubic Spline)

**Reference**: Moritz & Bartz-Beielstein (2017) - imputeTS: Time Series Missing Value Imputation

**Method**:
- Cubic spline interpolation for gaps
- Piecewise polynomial fitting

**Rationale**:
- Captures non-linear electricity price dynamics
- Avoids Runge's phenomenon (oscillation in polynomial interpolation)
- Smoother transitions than linear methods

### Step 3: Normalization

**Reference**: Faiq et al. (2024) - Impact of Data Normalization on Electricity Forecasting

**Methods**:
1. **Min-Max [0,1]**: For ML model inputs (LSTM optimal: CVRMSE 10.3)
2. **Z-Score**: For relative price position analysis

**Rationale**:
- Min-max proven optimal for neural network electricity forecasting
- Z-score useful for understanding relative market positions

In [ ]:
def detect_outliers_zscore(series, threshold=3.0):
    """
    Z-score outlier detection
    
    Reference: Chikodili et al. (2021)
    Threshold: 3.0 standard deviations (F-measure 0.9978)
    """
    z_scores = np.abs(zscore(series, nan_policy='omit'))
    return z_scores > threshold


def interpolate_spline(series):
    """
    Cubic spline interpolation for missing values
    
    Reference: Moritz & Bartz-Beielstein (2017) - imputeTS
    """
    if series.isna().sum() == 0:
        return series
    
    return series.interpolate(
        method='cubic',
        limit_direction='both',
        order=3
    )


def preprocess_zone_data(df, zone_name):
    """
    Complete preprocessing pipeline
    
    Steps:
    1. Extract date/hour from timestamp
    2. Detect and remove outliers (z-score)
    3. Interpolate missing values (cubic spline)
    
    Returns: Clean DataFrame and statistics
    """
    df_clean = df.copy()
    
    # Date and hour extraction
    df_clean['date'] = df_clean['timestamp'].dt.date
    df_clean['hour'] = df_clean['timestamp'].dt.hour
    
    # Step 1: Outlier detection
    outlier_mask = detect_outliers_zscore(df_clean['price_eur_mwh'])
    n_outliers = outlier_mask.sum()
    
    # Replace outliers with NaN
    df_clean.loc[outlier_mask, 'price_eur_mwh'] = np.nan
    
    # Step 2: Count original missing values
    n_missing_original = df_clean['price_eur_mwh'].isna().sum()
    
    # Step 3: Interpolate
    df_clean['price_eur_mwh'] = interpolate_spline(df_clean['price_eur_mwh'])
    
    # Statistics
    stats = {
        'zone': zone_name,
        'total_records': len(df_clean),
        'outliers_detected': n_outliers,
        'outlier_pct': round(100 * n_outliers / len(df_clean), 3),
        'missing_original': n_missing_original - n_outliers,
        'total_interpolated': n_missing_original
    }
    
    return df_clean, stats


print("Preprocessing functions defined")

In [ ]:
# Apply preprocessing
print("Applying preprocessing pipeline...\n")
print("Zone | Records   | Outliers | Missing | Interpolated")
print("-" * 60)

preprocessed_data = {}
preprocessing_stats = []

for zone, df in all_data.items():
    df_clean, stats = preprocess_zone_data(df, zone)
    preprocessed_data[zone] = df_clean
    preprocessing_stats.append(stats)
    
    # Save
    filepath = CLEAN_DIR / f"{zone}_clean.csv"
    df_clean.to_csv(filepath, index=False)
    
    print(f"{zone:4s} | {stats['total_records']:9,} | {stats['outliers_detected']:8d} | "
          f"{stats['missing_original']:7d} | {stats['total_interpolated']:12d}")

# Save statistics
stats_df = pd.DataFrame(preprocessing_stats)
stats_df.to_csv(DATA_DIR / 'preprocessing_statistics.csv', index=False)

print(f"\nCleaned data saved to {CLEAN_DIR}")
print(f"Statistics saved to preprocessing_statistics.csv")

## Daily Profile Creation

Transform hourly data into 24-hour daily profiles with dual normalization:

1. **Raw prices**: Original EUR/MWh values
2. **Min-Max [0,1]**: For clustering and ML models
3. **Z-Score**: For statistical analysis

Each profile represents one complete day (24 hours)

In [ ]:
def add_calendar_features(date):
    """
    Add temporal features for XGBoost model
    """
    dt = pd.to_datetime(date)
    
    weekday = dt.dayofweek
    is_weekend = weekday >= 5
    month = dt.month
    
    # Meteorological seasons
    if month in [12, 1, 2]:
        season = 'Winter'
    elif month in [3, 4, 5]:
        season = 'Spring'
    elif month in [6, 7, 8]:
        season = 'Summer'
    else:
        season = 'Autumn'
    
    return {
        'weekday': weekday,
        'is_weekend': is_weekend,
        'month': month,
        'season': season
    }


def create_daily_profiles(df, zone_name):
    """
    Create 24-hour daily profiles with triple representation:
    - Raw prices (EUR/MWh)
    - Min-max normalized [0,1]
    - Z-score normalized
    
    Output format:
    - Columns: h00_raw, h01_raw, ..., h23_raw (raw prices)
    - Columns: h00_minmax, h01_minmax, ..., h23_minmax (min-max normalized)
    - Columns: h00_zscore, h01_zscore, ..., h23_zscore (z-score normalized)
    - Plus: daily statistics and calendar features
    """
    # Pivot to 24-hour format
    profiles = df.pivot_table(
        index='date',
        columns='hour',
        values='price_eur_mwh',
        aggfunc='mean'
    )
    
    # Keep only complete days
    profiles = profiles.dropna()
    
    # Rename to raw columns
    raw_cols = {h: f'h{h:02d}_raw' for h in range(24)}
    profiles = profiles.rename(columns=raw_cols)
    
    # Get list of raw column names
    raw_cols_list = [f'h{h:02d}_raw' for h in range(24)]
    daily_values = profiles[raw_cols_list].values
    
    # Min-max normalization (per day)
    daily_min = daily_values.min(axis=1, keepdims=True)
    daily_max = daily_values.max(axis=1, keepdims=True)
    daily_range = daily_max - daily_min
    daily_range[daily_range == 0] = 1  # Avoid division by zero
    
    minmax_values = (daily_values - daily_min) / daily_range
    
    for h in range(24):
        profiles[f'h{h:02d}_minmax'] = minmax_values[:, h]
    
    # Z-score normalization (per day)
    zscore_values = zscore(daily_values, axis=1)
    
    for h in range(24):
        profiles[f'h{h:02d}_zscore'] = zscore_values[:, h]
    
    # Daily statistics
    profiles['daily_mean'] = daily_values.mean(axis=1)
    profiles['daily_std'] = daily_values.std(axis=1)
    profiles['daily_min'] = daily_values.min(axis=1)
    profiles['daily_max'] = daily_values.max(axis=1)
    profiles['daily_range'] = profiles['daily_max'] - profiles['daily_min']
    
    # Calendar features
    cal_features = profiles.index.map(add_calendar_features)
    profiles['weekday'] = [f['weekday'] for f in cal_features]
    profiles['is_weekend'] = [f['is_weekend'] for f in cal_features]
    profiles['month'] = [f['month'] for f in cal_features]
    profiles['season'] = [f['season'] for f in cal_features]
    
    # Zone metadata
    profiles['zone'] = zone_name
    profiles['country'] = COUNTRY_MAP[zone_name]
    
    return profiles


print("Creating daily profiles with dual normalization...\n")
all_profiles = {}

for zone, df in preprocessed_data.items():
    prof = create_daily_profiles(df, zone)
    all_profiles[zone] = prof
    
    # Save as Parquet (efficient for large datasets)
    filepath = PROFILES_DIR / f"{zone}_profiles.parquet"
    prof.to_parquet(filepath)
    
    print(f"{zone}: {len(prof)} complete days -> {filepath.name}")

print(f"\nAll profiles saved to {PROFILES_DIR}")
print(f"\nTotal complete days: {sum(len(p) for p in all_profiles.values())}")

## Quality Assessment

In [ ]:
def generate_quality_table():
    """
    Generate quality metrics table for proposal/report
    """
    rows = []
    
    for zone in ZONES:
        if zone not in all_profiles:
            continue
        
        prof = all_profiles[zone]
        stats = next(s for s in preprocessing_stats if s['zone'] == zone)
        
        row = {
            'Zone': zone,
            'Country': COUNTRY_MAP[zone],
            'Complete Days': len(prof),
            'Outliers Removed': stats['outliers_detected'],
            'Outlier Rate (%)': stats['outlier_pct'],
            'Values Interpolated': stats['total_interpolated'],
            'Mean Price (EUR/MWh)': round(prof['daily_mean'].mean(), 2),
            'Std Price (EUR/MWh)': round(prof['daily_mean'].std(), 2)
        }
        rows.append(row)
    
    return pd.DataFrame(rows)


quality_table = generate_quality_table()
quality_table.to_csv(DATA_DIR / 'quality_metrics.csv', index=False)

print("Quality Metrics:")
print(quality_table.to_string(index=False))
print(f"\nSaved to: {DATA_DIR / 'quality_metrics.csv'}")

## Visualization: Price Distributions

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

countries = ['Spain', 'Poland', 'Norway', 'Sweden', 'Denmark']
all_prof = pd.concat([p for p in all_profiles.values()], ignore_index=False)

for idx, country in enumerate(countries):
    ax = axes[idx]
    country_data = all_prof[all_prof['country'] == country]
    
    ax.hist(country_data['daily_mean'], bins=50, alpha=0.7, edgecolor='black', color='steelblue')
    ax.axvline(country_data['daily_mean'].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
    ax.set_title(f'{country} (n={len(country_data)} days)', fontsize=12, fontweight='bold')
    ax.set_xlabel('Mean Daily Price (EUR/MWh)')
    ax.set_ylabel('Frequency')
    ax.legend()
    ax.grid(alpha=0.3)

axes[-1].axis('off')

plt.suptitle('Daily Mean Price Distributions (After Preprocessing)',
             fontsize=14, fontweight='bold')
plt.tight_layout()

filepath = FIGURES_DIR / 'price_distributions.png'
plt.savefig(filepath, dpi=300, bbox_inches='tight')
plt.close()

print(f"Figure saved: {filepath}")

## Visualization: Normalized Profiles Sample

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for idx, country in enumerate(countries):
    ax = axes[idx]
    country_data = all_prof[all_prof['country'] == country]
    
    # Sample 30 random days
    sample = country_data.sample(min(30, len(country_data)))
    
    minmax_cols = [f'h{h:02d}_minmax' for h in range(24)]
    
    # Plot individual profiles
    for _, row in sample.iterrows():
        ax.plot(range(24), row[minmax_cols], alpha=0.3, color='blue', linewidth=1)
    
    # Plot median
    median_profile = country_data[minmax_cols].median()
    ax.plot(range(24), median_profile, color='red', linewidth=3, label='Median')
    
    ax.set_title(f'{country}', fontsize=12, fontweight='bold')
    ax.set_xlabel('Hour of Day')
    ax.set_ylabel('Normalized Price [0,1]')
    ax.set_ylim([-0.05, 1.05])
    ax.set_xticks(range(0, 24, 4))
    ax.grid(alpha=0.3)
    ax.legend()

axes[-1].axis('off')

plt.suptitle('Min-Max Normalized Daily Profiles (30 samples + median)',
             fontsize=14, fontweight='bold')
plt.tight_layout()

filepath = FIGURES_DIR / 'normalized_profiles.png'
plt.savefig(filepath, dpi=300, bbox_inches='tight')
plt.close()

print(f"Figure saved: {filepath}")

## Summary and Next Steps

In [ ]:
print("="*70)
print("PREPROCESSING COMPLETE")
print("="*70)

print(f"\nZones processed: {len(all_profiles)}")
print(f"Total complete days: {sum(len(p) for p in all_profiles.values())}")

print(f"\nOutputs:")
print(f"  - Cleaned data: {CLEAN_DIR}")
print(f"  - Daily profiles: {PROFILES_DIR}")
print(f"  - Quality metrics: quality_metrics.csv")
print(f"  - Figures: {FIGURES_DIR}")

print(f"\nReady for:")
print(f"  1. Hierarchical Clustering (Ward's method)")
print(f"  2. XGBoost Classification (cluster prediction)")
print(f"  3. XGBoost Regression (hourly price prediction per cluster)")

print(f"\nData format:")
print(f"  - Each profile: 24h vector")
print(f"  - Normalization: Min-max [0,1] for clustering")
print(f"  - Features: Calendar (weekday, season, month)")
print(f"  - Raw prices: Available for final predictions")